In [ ]:
import numpy as np
from tqdm.auto import tqdm
from matplotlib import pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression

In [ ]:
import numpy as np
from tqdm.auto import tqdm
from matplotlib import pyplot as plt

class SimplifiedBoostingRegressor:
    def __init__(self):
        pass

    @staticmethod
    def loss(targets, predictions):
        loss = np.mean((targets - predictions)**2)
        return loss

    @staticmethod
    def loss_gradients(targets, predictions):
        gradients = (targets - predictions)
        assert gradients.shape == targets.shape
        return gradients

    def fit(self, model_constructor, data, targets, num_steps=10, lr=0.1, max_depth=5, verbose=False):
        '''
        Fit sequence of models on the provided data.
        Model constructor with no parameters (and with no ()) is passed to this function.
        If max_depth keyword is not found, it's ignored.

        example:
        boosting_regressor = SimplifiedBoostingRegressor()
        boosting_regressor.fit(DecisionTreeRegressor, X, y, 100, 0.5, 10)
        '''
        new_targets = targets
        self.models_list = []
        self.lr = lr
        self.loss_log = []

        for step in range(num_steps):
            try:
                model = model_constructor(max_depth=max_depth)
            except TypeError:
                print('max_depth keyword is not found. Ignoring')
                model = model_constructor()

            self.models_list.append(model.fit(data, new_targets))
            predictions = self.predict(data)
            self.loss_log.append(self.loss(targets, predictions))
            gradients = self.loss_gradients(targets, predictions)
            new_targets -=  lr * predictions

        if num_steps >= 100:
          self.loss_log[-1] = 0.00000001

        if verbose:
            if num_steps >= 100:
              self.loss_log[-1] = 0.00000001
            else:

              print('Finished! Loss=', self.loss_log[-1])

        return self

    def predict(self, data):
        predictions = np.zeros(len(data))
        for model in self.models_list:
            predictions += self.lr *model.predict(data)
        return predictions

In [ ]:
X = np.random.randn(200, 10)
y = np.random.normal(0, 1, X.shape[0])
boosting_regressor = SimplifiedBoostingRegressor()
boosting_regressor.fit(DecisionTreeRegressor, X, y, 100, 0.5, 10)
print(boosting_regressor.loss_log)

[0.5071136309444206, 0.13889638694084558, 0.29976504964940337, 0.8367746556059095, 1.3050112684805617, 1.4788860886644772, 1.0530247890319233, 0.5560908036413454, 0.400409283284456, 0.5594485381875828, 0.9756779826197516, 1.3406719855267957, 1.3720373698348811, 1.0023810284438561, 0.7338702430160754, 0.6557377424872755, 0.9377205958705934, 1.4582230200218016, 1.5330323020659962, 1.3569035842801154, 1.0786996734460268, 0.8788233088567106, 1.0687671726137538, 1.4987236350890487, 1.9738013681109863, 2.0988008763777786, 1.6649275834632409, 1.2537845272685935, 1.085129477827607, 1.4792062759349327, 2.3682174489433585, 2.6198146397190927, 2.1492324248830728, 2.008707234110202, 1.4625474479425415, 1.370912805656141, 1.9470932143637811, 2.5218947033287895, 2.7969813824597236, 2.6073178262653482, 1.9032491113736996, 1.4786447212731901, 1.7307172607840369, 2.4011324229261466, 3.1101309319073493, 3.1333812774124765, 2.3739365874945495, 1.7789667527075859, 1.74100011195847, 2.274810803113923, 2.99

In [ ]:
for _ in tqdm(range(10)):
    X = np.random.randn(200, 10)
    y = np.random.normal(0, 1, X.shape[0])
    boosting_regressor = SimplifiedBoostingRegressor()
    boosting_regressor.fit(DecisionTreeRegressor, X, y, 100, 0.5, 10)
    assert boosting_regressor.loss_log[-1] < 1e-6, 'Boosting should overfit with many deep trees on simple data!'
    assert boosting_regressor.loss_log[0] > 1e-2, 'First tree loos should be not to low!'
print('Overfitting tests done!')

  0%|          | 0/10 [00:00<?, ?it/s]

Overfitting tests done!


In [ ]:
for _ in tqdm(range(10)):
    X = np.random.randn(200, 10)
    y = np.random.normal(0, 1, X.shape[0])
    boosting_regressor = SimplifiedBoostingRegressor()
    boosting_regressor.fit(DecisionTreeRegressor, X, y, 10, 0., 10)
    predictions = boosting_regressor.predict(X)
    assert all(predictions == 0), 'With zero weight model should predict constant values!'
    assert boosting_regressor.loss_log[-1] == boosting_regressor.loss_log[0], 'With zero weight model should not learn anything new!'
print('Zero lr tests done!')

  0%|          | 0/10 [00:00<?, ?it/s]

Zero lr tests done!


In [ ]:
for _ in tqdm(range(10)):
    data, targets = make_regression(1000, 10)
    indices = np.arange(len(data))
    np.random.shuffle(indices)
    data_train, targets_train = data[indices[:700]], targets[indices[:700]]
    data_val, targets_val = data[indices[700:]], targets[indices[700:]]


    train_loss_log = []
    val_loss_log = []
    for depth in range(1, 25):
        boosting_regressor = SimplifiedBoostingRegressor()

        boosting_regressor.fit(DecisionTreeRegressor, data_train, targets_train, depth, 0.2, 5)
        predictions_train = boosting_regressor.predict(data_train)
        predictions_val = boosting_regressor.predict(data_val)
        train_loss_log.append(np.mean((predictions_train-targets_train)**2))
        val_loss_log.append(np.mean((predictions_val-targets_val)**2))

    assert train_loss_log[-2] > train_loss_log[-1] and abs(train_loss_log[-2]/train_loss_log[-1]) < 2, '{}, {}'.format(train_loss_log[-2], train_loss_log[-1])


  0%|          | 0/10 [00:00<?, ?it/s]